# demo 4 - kernel

Compare performances of models based on 
1. fixed kernel
2. the regular
3. update kernel after every step of forward pass


In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from iHMM_GP.step1_utils import *
from iHMM_GP.data_setup import *
#do not use these variable names: f0, f1, f2, g1, g2, g3, g5, f_new1, f_new2, f_new3

T = 100
n = 50
sigma2 = 2

T_test = 50

# create data
data, s = sim_new_data2(f_true, Pi_true, T=T+T_test, n=n, sigma2=sigma2)

# get training data - for t in tau, hold out q% of the data
data_train = []

for t in range(T):
    X, Y = data[t]
    data_train.append((X, Y))
    
# test 3 - new sequence state prediction
data_test3 = data[T:]
    
# state labels:
s_train_true = s[0:T]
s_test_true = s[T:]

In [9]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from iHMM_GP.gp_utils import *

In [10]:
iHMM_params = (3,3,3)

x, y = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
Z = np.array([[x[i, j], y[i, j]] for i in range(10) for j in range(10)])

from sklearn.metrics import rand_score, adjusted_mutual_info_score, v_measure_score
import time
import pickle

In [12]:
# fixed kernel
kernel_bounds = [(1,1), (1,1)]
noise_bounds = (1e-3, 1e3)
time0 = time.time()

marginal_lls, sigma2, all_kernels = get_all_GPs(data_train, k_ls_bounds=kernel_bounds[1], k_var_bounds=kernel_bounds[0])
#marginal_lls, sigma2, all_kernels = get_all_GPs(data_train)
k_params = [(1,1), (1,1)] 

s1, o1, models1, N1, M1 = forward_pass(data_train, iHMM_params, marginal_lls, all_kernels, sigma2, Z)
s2, o2, models2, N2, M2 = refinement(s1, o1, N1, M1, models1, iHMM_params, sigma2, marginal_lls, data_train, min_cluster_size=3, verbose=False)

logliks_train = get_likelihoods(data_train, models2, sigma2); logliks_train = np.array(logliks_train)
Pi_est, _ = get_nm(s2, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
Pi_hat = Pi_est / row_sums[:, np.newaxis]
s_final_train = viterbi(None, None, Pi_hat, logliks_train)

# check if any cluster has 0 time points, can remove it
s_final_train = rename_s(s_final_train)[1]
K_opt = len(np.unique(s_final_train))

final_models = get_final_results(data_train, s_final_train, sigma2, Z, n_jobs=-1, N_max=500) #new 2 lines

time0 =time.time() - time0
logliks_train = get_likelihoods(data_train, final_models, sigma2); logliks_train = np.array(logliks_train) #
Pi_est, _ = get_nm(s_final_train, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
Pi_hat = Pi_est / row_sums[:, np.newaxis]
logliks_test = get_likelihoods(data_test3, final_models, sigma2); logliks_test = np.array(logliks_test)
s_final_test = viterbi(None, None, Pi_hat, logliks_test)

hmm_loglik_train = compute_likelihood_HMM(Pi_hat, logliks_train)
hmm_loglik_test = compute_likelihood_HMM(Pi_hat, logliks_test)


train_label_acc = (rand_score(s_train_true, s_final_train), adjusted_mutual_info_score(s_train_true, s_final_train), v_measure_score(s_train_true, s_final_train))
test_label_acc = (rand_score(s_test_true, s_final_test), adjusted_mutual_info_score(s_test_true, s_final_test), v_measure_score(s_test_true, s_final_test))

obj = {'K': K_opt,
     'hmm_loglik_train': hmm_loglik_train,
     'hmm_loglik_test': hmm_loglik_test,
     'train_label_accuracy': train_label_acc,
     'test_label_accuracy': test_label_acc,
     'time': time0}

In [13]:
obj

{'K': 4,
 'hmm_loglik_train': -18303.26485526144,
 'hmm_loglik_test': -9184.423412348819,
 'train_label_accuracy': (0.6513131313131313,
  0.35966459591530375,
  0.4124099641822997),
 'test_label_accuracy': (0.603265306122449,
  0.4648899009259502,
  0.5544489247621441),
 'time': 13.591920852661133}

In [16]:
# regular
# fixed kernel

#kernel_bounds = [(1e-3, 1e3), (1e-3,1e3)]
#noise_bounds = (1e-3, 1e3)
time0 = time.time()

#marginal_lls, sigma2, all_kernels = get_all_GPs(data_train, k_ls_bounds=kernel_bounds[1], k_var_bounds=kernel_bounds[0])
marginal_lls, sigma2, all_kernels = get_all_GPs(data_train)
#k_params = [(1,1), (1,1)] 

s1, o1, models1, N1, M1 = forward_pass(data_train, iHMM_params, marginal_lls, all_kernels, sigma2, Z, update_kernel_every=200)
s2, o2, models2, N2, M2 = refinement(s1, o1, N1, M1, models1, iHMM_params, sigma2, marginal_lls, data_train, min_cluster_size=2, verbose=False)

logliks_train = get_likelihoods(data_train, models2, sigma2); logliks_train = np.array(logliks_train)
Pi_est, _ = get_nm(s2, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
Pi_hat = Pi_est / row_sums[:, np.newaxis]
s_final_train = viterbi(None, None, Pi_hat, logliks_train)

# check if any cluster has 0 time points, can remove it
s_final_train = rename_s(s_final_train)[1]
K_opt = len(np.unique(s_final_train))

final_models = get_final_results(data_train, s_final_train, sigma2, Z, n_jobs=4, N_max=500) #new 2 lines

time0 =time.time() - time0
logliks_train = get_likelihoods(data_train, final_models, sigma2); logliks_train = np.array(logliks_train) #
Pi_est, _ = get_nm(s_final_train, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
Pi_hat = Pi_est / row_sums[:, np.newaxis]
logliks_test = get_likelihoods(data_test3, final_models, sigma2); logliks_test = np.array(logliks_test)
s_final_test = viterbi(None, None, Pi_hat, logliks_test)

hmm_loglik_train = compute_likelihood_HMM(Pi_hat, logliks_train)
hmm_loglik_test = compute_likelihood_HMM(Pi_hat, logliks_test)


train_label_acc = (rand_score(s_train_true, s_final_train), adjusted_mutual_info_score(s_train_true, s_final_train), v_measure_score(s_train_true, s_final_train))
test_label_acc = (rand_score(s_test_true, s_final_test), adjusted_mutual_info_score(s_test_true, s_final_test), v_measure_score(s_test_true, s_final_test))

obj = {'K': K_opt,
     'hmm_loglik_train': hmm_loglik_train,
     'hmm_loglik_test': hmm_loglik_test,
     'train_label_accuracy': train_label_acc,
     'test_label_accuracy': test_label_acc,
     'time': time0}

In [15]:
obj

{'K': 7,
 'hmm_loglik_train': -17663.572826106414,
 'hmm_loglik_test': -9023.088120835848,
 'train_label_accuracy': (0.9278787878787879,
  0.7847737675212959,
  0.8152667237553921),
 'test_label_accuracy': (0.9493877551020408,
  0.8659202870903,
  0.9037584753685233),
 'time': 28.077348709106445}

In [6]:
# update every 10
# fixed kernel
kernel_bounds = [(1e-3, 1e3), (1e-3,1e3)]
noise_bounds = (1e-3, 1e3)
time0 = time.time()

marginal_lls, sigma2, all_kernels = get_all_GPs(data_train, k_ls_bounds=kernel_bounds[1], k_var_bounds=kernel_bounds[0])
k_params = [(1,1), (1,1)] 

s1, o1, models1, N1, M1 = forward_pass(data_train, iHMM_params, marginal_lls, all_kernels, sigma2, Z, update_kernel_every=5)
s2, o2, models2, N2, M2 = refinement(s1, o1, N1, M1, models1, iHMM_params, sigma2, marginal_lls, data_train, min_cluster_size=3, verbose=False)

logliks_train = get_likelihoods(data_train, models2, sigma2); logliks_train = np.array(logliks_train)
Pi_est, _ = get_nm(s2, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
Pi_hat = Pi_est / row_sums[:, np.newaxis]
s_final_train = viterbi(None, None, Pi_hat, logliks_train)

# check if any cluster has 0 time points, can remove it
s_final_train = rename_s(s_final_train)[1]
K_opt = len(np.unique(s_final_train))

final_models = get_final_results(data_train, s_final_train, sigma2, Z, n_jobs=4, N_max=500) #new 2 lines

time0 =time.time() - time0
logliks_train = get_likelihoods(data_train, final_models, sigma2); logliks_train = np.array(logliks_train) #
Pi_est, _ = get_nm(s_final_train, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
Pi_hat = Pi_est / row_sums[:, np.newaxis]
logliks_test = get_likelihoods(data_test3, final_models, sigma2); logliks_test = np.array(logliks_test)
s_final_test = viterbi(None, None, Pi_hat, logliks_test)

hmm_loglik_train = compute_likelihood_HMM(Pi_hat, logliks_train)
hmm_loglik_test = compute_likelihood_HMM(Pi_hat, logliks_test)


train_label_acc = (rand_score(s_train_true, s_final_train), adjusted_mutual_info_score(s_train_true, s_final_train), v_measure_score(s_train_true, s_final_train))
test_label_acc = (rand_score(s_test_true, s_final_test), adjusted_mutual_info_score(s_test_true, s_final_test), v_measure_score(s_test_true, s_final_test))

obj = {'K': K_opt,
     'hmm_loglik_train': hmm_loglik_train,
     'hmm_loglik_test': hmm_loglik_test,
     'train_label_accuracy': train_label_acc,
     'test_label_accuracy': test_label_acc,
     'time': time0}

In [7]:
obj

{'K': 8,
 'hmm_loglik_train': -17907.732061929346,
 'hmm_loglik_test': -9089.432503732034,
 'train_label_accuracy': (0.8840404040404041,
  0.6237863119189498,
  0.6810117918112812),
 'test_label_accuracy': (0.893061224489796,
  0.5850042362141471,
  0.7036056191633112),
 'time': 88.92849802970886}

In [17]:
def simul(T=100, n=50, sigma2=2):
    
    # set up data
    T_test = 50

    # create data
    data, s = sim_new_data2(f_true, Pi_true, T=T+T_test, n=n, sigma2=sigma2)

    # get training data - for t in tau, hold out q% of the data
    data_train = []

    for t in range(T):
        X, Y = data[t]
        data_train.append((X, Y))

    # test 3 - new sequence state prediction
    data_test3 = data[T:]

    # state labels:
    s_train_true = s[0:T]
    s_test_true = s[T:]
    
    iHMM_params = (3,3,3)

    x, y = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
    Z = np.array([[x[i, j], y[i, j]] for i in range(10) for j in range(10)])

    from sklearn.metrics import rand_score, adjusted_mutual_info_score, v_measure_score
    import time
    
    ## METHOD 1
    kernel_bounds = [(1,1), (1,1)]
    noise_bounds = (1e-3, 1e3)
    time0 = time.time()

    marginal_lls, sigma2, all_kernels = get_all_GPs(data_train, k_ls_bounds=kernel_bounds[1], k_var_bounds=kernel_bounds[0])
    k_params = [(1,1), (1,1)] 

    s1, o1, models1, N1, M1 = forward_pass(data_train, iHMM_params, marginal_lls, all_kernels, sigma2, Z, update_kernel_every=200)
    s2, o2, models2, N2, M2 = refinement(s1, o1, N1, M1, models1, iHMM_params, sigma2, marginal_lls, data_train, min_cluster_size=3, verbose=False)

    logliks_train = get_likelihoods(data_train, models2, sigma2); logliks_train = np.array(logliks_train)
    Pi_est, _ = get_nm(s2, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    s_final_train = viterbi(None, None, Pi_hat, logliks_train)

    # check if any cluster has 0 time points, can remove it
    s_final_train = rename_s(s_final_train)[1]
    K_opt = len(np.unique(s_final_train))

    final_models = get_final_results(data_train, s_final_train, sigma2, Z, n_jobs=-1, N_max=500) #new 2 lines

    time0 =time.time() - time0
    logliks_train = get_likelihoods(data_train, final_models, sigma2); logliks_train = np.array(logliks_train) #
    Pi_est, _ = get_nm(s_final_train, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    logliks_test = get_likelihoods(data_test3, final_models, sigma2); logliks_test = np.array(logliks_test)
    s_final_test = viterbi(None, None, Pi_hat, logliks_test)

    hmm_loglik_train = compute_likelihood_HMM(Pi_hat, logliks_train)
    hmm_loglik_test = compute_likelihood_HMM(Pi_hat, logliks_test)


    train_label_acc = (rand_score(s_train_true, s_final_train), adjusted_mutual_info_score(s_train_true, s_final_train), v_measure_score(s_train_true, s_final_train))
    test_label_acc = (rand_score(s_test_true, s_final_test), adjusted_mutual_info_score(s_test_true, s_final_test), v_measure_score(s_test_true, s_final_test))

    obj1 = {'K': K_opt,
         'hmm_loglik_train': hmm_loglik_train,
         'hmm_loglik_test': hmm_loglik_test,
         'train_label_accuracy': train_label_acc,
         'test_label_accuracy': test_label_acc,
         'time': time0}

    ## METHOD 2
    #kernel_bounds = [(1e-3, 1e3), (1e-3,1e3)]
    #noise_bounds = (1e-3, 1e3)
    time0 = time.time()

    marginal_lls, sigma2, all_kernels = get_all_GPs(data_train)
    k_params = [(1,1), (1,1)] 

    s1, o1, models1, N1, M1 = forward_pass(data_train, iHMM_params, marginal_lls, all_kernels, sigma2, Z, update_kernel_every=200)
    s2, o2, models2, N2, M2 = refinement(s1, o1, N1, M1, models1, iHMM_params, sigma2, marginal_lls, data_train, min_cluster_size=3, verbose=False)

    logliks_train = get_likelihoods(data_train, models2, sigma2); logliks_train = np.array(logliks_train)
    Pi_est, _ = get_nm(s2, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    s_final_train = viterbi(None, None, Pi_hat, logliks_train)

    # check if any cluster has 0 time points, can remove it
    s_final_train = rename_s(s_final_train)[1]
    K_opt = len(np.unique(s_final_train))

    final_models = get_final_results(data_train, s_final_train, sigma2, Z, n_jobs=-1, N_max=500) #new 2 lines

    time0 =time.time() - time0
    logliks_train = get_likelihoods(data_train, final_models, sigma2); logliks_train = np.array(logliks_train) #
    Pi_est, _ = get_nm(s_final_train, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    logliks_test = get_likelihoods(data_test3, final_models, sigma2); logliks_test = np.array(logliks_test)
    s_final_test = viterbi(None, None, Pi_hat, logliks_test)

    hmm_loglik_train = compute_likelihood_HMM(Pi_hat, logliks_train)
    hmm_loglik_test = compute_likelihood_HMM(Pi_hat, logliks_test)


    train_label_acc = (rand_score(s_train_true, s_final_train), adjusted_mutual_info_score(s_train_true, s_final_train), v_measure_score(s_train_true, s_final_train))
    test_label_acc = (rand_score(s_test_true, s_final_test), adjusted_mutual_info_score(s_test_true, s_final_test), v_measure_score(s_test_true, s_final_test))

    obj2 = {'K': K_opt,
         'hmm_loglik_train': hmm_loglik_train,
         'hmm_loglik_test': hmm_loglik_test,
         'train_label_accuracy': train_label_acc,
         'test_label_accuracy': test_label_acc,
         'time': time0}
    
    ## METHOD 3
    #kernel_bounds = [(1e-3, 1e3), (1e-3,1e3)]
    #noise_bounds = (1e-3, 1e3)
    time0 = time.time()

    marginal_lls, sigma2, all_kernels = get_all_GPs(data_train)
    k_params = [(1,1), (1,1)] 

    s1, o1, models1, N1, M1 = forward_pass(data_train, iHMM_params, marginal_lls, all_kernels, sigma2, Z, update_kernel_every=5)
    s2, o2, models2, N2, M2 = refinement(s1, o1, N1, M1, models1, iHMM_params, sigma2, marginal_lls, data_train, min_cluster_size=3, verbose=False)

    logliks_train = get_likelihoods(data_train, models2, sigma2); logliks_train = np.array(logliks_train)
    Pi_est, _ = get_nm(s2, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    s_final_train = viterbi(None, None, Pi_hat, logliks_train)

    # check if any cluster has 0 time points, can remove it
    s_final_train = rename_s(s_final_train)[1]
    K_opt = len(np.unique(s_final_train))

    final_models = get_final_results(data_train, s_final_train, sigma2, Z, n_jobs=-1, N_max=500) #new 2 lines

    time0 =time.time() - time0
    logliks_train = get_likelihoods(data_train, final_models, sigma2); logliks_train = np.array(logliks_train) #
    Pi_est, _ = get_nm(s_final_train, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    logliks_test = get_likelihoods(data_test3, final_models, sigma2); logliks_test = np.array(logliks_test)
    s_final_test = viterbi(None, None, Pi_hat, logliks_test)

    hmm_loglik_train = compute_likelihood_HMM(Pi_hat, logliks_train)
    hmm_loglik_test = compute_likelihood_HMM(Pi_hat, logliks_test)


    train_label_acc = (rand_score(s_train_true, s_final_train), adjusted_mutual_info_score(s_train_true, s_final_train), v_measure_score(s_train_true, s_final_train))
    test_label_acc = (rand_score(s_test_true, s_final_test), adjusted_mutual_info_score(s_test_true, s_final_test), v_measure_score(s_test_true, s_final_test))

    obj3 = {'K': K_opt,
         'hmm_loglik_train': hmm_loglik_train,
         'hmm_loglik_test': hmm_loglik_test,
         'train_label_accuracy': train_label_acc,
         'test_label_accuracy': test_label_acc,
         'time': time0}
    return (obj1, obj2, obj3)

In [20]:
simul(T=120, n=50, sigma2=2)

({'K': 5,
  'hmm_loglik_train': -21909.939800983,
  'hmm_loglik_test': -9206.177934056244,
  'train_label_accuracy': (0.6509803921568628,
   0.44387932941182623,
   0.4953250485457449),
  'test_label_accuracy': (0.6277551020408163,
   0.4677119017122544,
   0.5608551034233984),
  'time': 9.68434190750122},
 {'K': 10,
  'hmm_loglik_train': -21093.391501214202,
  'hmm_loglik_test': -8969.08983115837,
  'train_label_accuracy': (0.9592436974789916,
   0.8511422246088863,
   0.8742497251176026),
  'test_label_accuracy': (0.9918367346938776,
   0.962335553197468,
   0.9753099470389273),
  'time': 35.32162404060364},
 {'K': 8,
  'hmm_loglik_train': -21087.443041259787,
  'hmm_loglik_test': -8922.153469472501,
  'train_label_accuracy': (0.9572829131652661,
   0.88401066078267,
   0.8993795812045308),
  'test_label_accuracy': (0.9755102040816327,
   0.9476564313664771,
   0.964148938177539),
  'time': 87.07919692993164})

In [21]:
results = []
rep = 30

for i in range(rep):
    print('working at ', i)
    res = simul(T=120, n=50, sigma2=2)
    results.append(res)
    
pickle.dump(results, open( "new_simulation_results/demo4.p", "wb" ) )

working at  0
working at  1
working at  2
working at  3
working at  4
working at  5
working at  6
working at  7
working at  8
working at  9
working at  10
working at  11
working at  12
working at  13
working at  14
working at  15
working at  16
working at  17
working at  18
working at  19
working at  20
working at  21
working at  22
working at  23
working at  24
working at  25
working at  26
working at  27
working at  28
working at  29


In [16]:
pickle.dump(results, open( "results/demo4.p", "wb" ) )